In [1]:
import pandas as pd
import numpy as np
import requests
import json

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Travail sur un jeu de donnée non équilibré
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline

from P7 import params
from P7 import preprocess

import pickle

In [4]:
df = pd.read_csv("../" + params.PATH_OUTPUT + "new_application.csv",
                 nrows = 10_000)
df.head()


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train_df = df[df["TARGET"].notnull()]
test_df = df[df["TARGET"].isnull()]

# Create train labels
train_labels = df["TARGET"].astype(int)
train = df.drop(columns="TARGET").copy()

# Drop the IDs and record them for later
train_ids = train["SK_ID_CURR"]
train.drop(columns = ["SK_ID_CURR"], inplace = True)

train.replace([np.inf, -np.inf], np.nan, inplace=True)
train.dropna(axis=1, how="all", inplace=True)

# # Feature names
features = list(train.columns)

In [6]:
# Description des données de chaque fichier
from P7.params import PATH_INPUT


file = "HomeCredit_columns_description.csv"
# os.path.exists(path)

column_description = pd.read_csv("../"+PATH_INPUT + file, encoding="cp1252")
print(column_description.shape)
print(column_description.columns)
print("Row duplicated:", column_description.Row.duplicated().sum())

(219, 5)
Index(['Unnamed: 0', 'Table', 'Row', 'Description', 'Special'], dtype='object')
Row duplicated: 23


In [20]:
feat_imp = pd.read_csv("../" + "output/" +
                       "feature_selection_application.csv")
feat_imp[:10]
# list(feat_imp["feature"][:10])

,feature,importance
0,EXT_SOURCE_3,0.041073
1,OBS_60_CNT_SOCIAL_CIRCLE,0.030548
2,EXT_SOURCE_2,0.027819
3,OBS_30_CNT_SOCIAL_CIRCLE,0.025452
4,AMT_REQ_CREDIT_BUREAU_YEAR,0.022505
5,CNT_CHILDREN,0.020741
6,CNT_FAM_MEMBERS,0.020252
7,EXT_SOURCE_1,0.020170
8,PAYMENT_RATE,0.020046
9,FLAG_PHONE,0.019548


In [ ]:
# for feature in list(feat_imp["feature"][:10]):
#     print(column_description[column_description["Row"] == feature][["Row", "Description"]])

In [92]:
pd.read_csv("../output/perf_model.csv")

,model,score,train_score,test_score
0,Logistic Regression standard,F-Beta,0.003381,0.005299
1,Logistic Regression SMOTE,F-Beta,0.381847,0.390972
2,Logistic Regression SMOTETOMEK,F-Beta,0.382531,0.400398
3,Logistic Regression SMOTEENN,F-Beta,0.374104,0.385799
4,xgb SMOTE,F-Beta,0.530778,0.192391
5,LGBM SMOTE,F-Beta,0.400074,0.200087
6,rfc SMOTE,F-Beta,0.938454,0.186147


In [2]:
model = pd.read_pickle(r"../model/LR_SMOTE.pkl")

In [7]:
list(feat_imp[:10]["feature"])

['EXT_SOURCE_3',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'EXT_SOURCE_2',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'PAYMENT_RATE',
 'FLAG_PHONE']

In [8]:
indiv_1 = pd.read_csv("../output/new_trainScaled_application.csv")

In [95]:
indiv_1.columns

Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT', 'SK_ID_CURR', 'TARGET'],
      dtype='object', length=755)

In [21]:
col = list(feat_imp[:10]["feature"])
# col.append("SK_ID_CURR")
col

['EXT_SOURCE_3',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'EXT_SOURCE_2',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'PAYMENT_RATE',
 'FLAG_PHONE']

In [15]:
indiv_1 = pd.read_csv("../output/new_trainScaled_application.csv",
                      index_col="SK_ID_CURR",
                      usecols=list(feat_imp[:10]["feature"]).append(
                          ["SK_ID_CURR", "TARGET"]))


In [11]:
indiv_1[indiv_1["TARGET"]==1]

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.0,0.0,0.0,0.000,0.001512,0.090287,0.088480,0.076404,0.253810,0.898118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100031,1.0,0.0,0.0,0.000,0.000742,0.233456,0.097762,0.164045,0.243084,0.368228,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100047,0.0,0.0,0.0,0.000,0.001512,0.286787,0.128826,0.202247,0.342216,0.439277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100049,1.0,0.0,1.0,0.000,0.000935,0.060892,0.055500,0.048315,0.094088,0.673703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
100096,1.0,0.0,0.0,0.000,0.000473,0.051685,0.048995,0.051685,0.390830,0.020994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123248,1.0,0.0,0.0,0.000,0.001127,0.044944,0.096303,0.044944,0.079194,0.644414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
123278,1.0,1.0,0.0,0.125,0.000935,0.033708,0.056467,0.033708,0.245585,0.918311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
123286,0.0,0.0,0.0,0.000,0.001512,0.089312,0.159415,0.084270,0.444988,0.110634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [12]:
indiv_1[indiv_1["TARGET"] == 0]


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100003,1.0,0.0,1.0,0.000,0.002089,0.311736,0.131445,0.270787,0.041792,0.480293,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100004,0.0,1.0,0.0,0.000,0.000358,0.022472,0.018353,0.022472,0.131976,0.349808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100006,1.0,0.0,0.0,0.000,0.000935,0.066837,0.107958,0.062921,0.104008,0.352154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100007,0.0,0.0,0.0,0.000,0.000819,0.116854,0.077404,0.116854,0.390830,0.299125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
100008,0.0,0.0,0.0,0.000,0.000627,0.111235,0.099485,0.102247,0.489878,0.470225,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123321,1.0,0.0,0.0,0.125,0.009398,0.078652,0.060387,0.078652,0.125266,0.820662,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
123322,0.0,1.0,0.0,0.000,0.001704,0.269706,0.178735,0.247191,0.306940,0.617585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
123323,1.0,1.0,0.0,0.125,0.000935,0.101124,0.074451,0.101124,0.359278,0.623591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [13]:
indiv_1.iloc[lambda x: x.index == 100002][list(feat_imp[:10]["feature"])]

,EXT_SOURCE_3,OBS_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_YEAR,CNT_CHILDREN,CNT_FAM_MEMBERS,EXT_SOURCE_1,PAYMENT_RATE,FLAG_PHONE
SK_ID_CURR,,,,,,,,,,
100002,0.156898,0.08,0.307507,0.08,0.04,0.0,0.0,0.06535,0.376626,1.0


In [14]:
indiv_1.iloc[lambda x: x.index == 100003][list(feat_imp[:10]["feature"])].to_dict()

{'EXT_SOURCE_3': {100003: 0.6042626548803929},
 'OBS_60_CNT_SOCIAL_CIRCLE': {100003: 0.04},
 'EXT_SOURCE_2': {100003: 0.7277592806510249},
 'OBS_30_CNT_SOCIAL_CIRCLE': {100003: 0.04},
 'AMT_REQ_CREDIT_BUREAU_YEAR': {100003: 0.0},
 'CNT_CHILDREN': {100003: 0.0},
 'CNT_FAM_MEMBERS': {100003: 0.1111111111111111},
 'EXT_SOURCE_1': {100003: 0.3162894358501382},
 'PAYMENT_RATE': {100003: 0.0521029941555441},
 'FLAG_PHONE': {100003: 1.0}}

In [6]:
# Valeur = 0
data_0 = {"EXT_SOURCE_3": 0.6042626548803929,
 "OBS_60_CNT_SOCIAL_CIRCLE": 0.04,
 "EXT_SOURCE_2": 0.7277592806510249,
 "OBS_30_CNT_SOCIAL_CIRCLE": 0.04,
 "AMT_REQ_CREDIT_BUREAU_YEAR": 0.0,
 "CNT_CHILDREN": 0.0,
 "CNT_FAM_MEMBERS": 0.1111111111111111,
 "EXT_SOURCE_1": 0.3162894358501382,
 "PAYMENT_RATE": 0.0521029941555441,
 "FLAG_PHONE": 1.0}

# valeur = 1
data_1 = {
    "EXT_SOURCE_3": 0.156898,
    "OBS_60_CNT_SOCIAL_CIRCLE": 0.08,
    "EXT_SOURCE_2": 0.307507,
    "OBS_30_CNT_SOCIAL_CIRCLE": 0.08,
    "AMT_REQ_CREDIT_BUREAU_YEAR": 0.04,
    "CNT_CHILDREN": 0,
    "CNT_FAM_MEMBERS": 0,
    "EXT_SOURCE_1": 0.06535,
    "PAYMENT_RATE": 0.376626,
    "FLAG_PHONE": 1
}

In [70]:
type(indiv_1.iloc[0])

pandas.core.series.Series

In [ ]:
result = []
for indiv in range(100002, 100100, 1):
    result.append(
        obj.predict(indiv_1.iloc[lambda x: x.index == 100002][col])[0])
result

In [107]:
obj.predict(indiv_1.iloc[lambda x: x.index == 100031][col])[0]

/home/moun_j/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


1

In [103]:
obj.predict(indiv_1.iloc[lambda x: x.index == 100002][col])[0]

/home/moun_j/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


1

In [104]:
obj.predict(indiv_1.iloc[lambda x: x.index == 100003][col])[0]

/home/moun_j/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0

In [105]:
obj.predict(indiv_1.iloc[lambda x: x.index == 100004][col])[0]

/home/moun_j/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0

## Requête de l"API

In [3]:
import os
# current_path = os.getcwd()
model_name = "LR_SMOTE.pkl"
credit_path = os.path.join("../API/ocp7api/", model_name)
with open(credit_path, "rb") as handle:
    model = pickle.load(handle)

In [4]:
# request-prediction
def request_prediction(model_uri, data):
    response = {}
    headers = {"Content-Type": "application/json"}

    print("---data : {}".format(type(data)))

    data_json = {"data": data}

    response = requests.request(method="POST",
                                headers=headers,
                                url=model_uri,
                                json=data_json)

    if response.status_code != 200:
        raise Exception("Request failed with status {}, {}".format(
            response.status_code, response.text))

    return response #json.load(response.content)["prediction"]


In [9]:
LRSMOTE_URI = "https://ocp7gitapi.herokuapp.com/predict"
# LRSMOTE_URI = "http://127.0.0.1:8000/predict"

In [10]:
result = json.loads(request_prediction(LRSMOTE_URI, data_0).content)
transparence = int(result["prediction"])
probability = result["probability"]
print(transparence, probability)

---data : <class 'dict'>


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb67d75cb50>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [8]:
result = json.loads(request_prediction(LRSMOTE_URI, data_1).content)
transparence = int(result["prediction"])
probability = result["probability"]
print(transparence, probability)

---data : <class 'dict'>
1 [[0.09307385 0.90692615]]


In [48]:
print(model.predict(indiv_1.iloc[lambda x: x.index == 100002][col]))
print(model.predict(indiv_1.iloc[lambda x: x.index == 100003][col]))


[1]
[0]


In [49]:
X = ((indiv_1.iloc[lambda x: x.index == 100002][col]).to_dict())

result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [50]:
X = ((indiv_1.iloc[lambda x: x.index == 100003][col]).to_dict())

result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [42]:
indiv_1.iloc[lambda x: x.index == 100002][col]

,EXT_SOURCE_3,OBS_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_YEAR,CNT_CHILDREN,CNT_FAM_MEMBERS,EXT_SOURCE_1,PAYMENT_RATE,FLAG_PHONE
SK_ID_CURR,,,,,,,,,,
100002,0.156898,0.08,0.307507,0.08,0.04,0.0,0.0,0.06535,0.376626,1.0


In [41]:
response = {}
headers = {"Content-Type": "application/json"}
data_json = {
    "data": (indiv_1.iloc[lambda x: x.index == 100002][col]).to_dict()
}

response = requests.request(method="POST",
                            headers=headers,
                            url=LRSMOTE_URI,
                            json=data_json)

response.content

b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [29]:
result = json.loads(request_prediction(LRSMOTE_URI, data_1).content)
transparence = int(result["prediction"])
probability = result["probability"]
print(transparence, probability)

---data : <class 'dict'>
1 [[0.09307385 0.90692615]]


In [183]:
result = json.loads(request_prediction(LRSMOTE_URI, data_0).content)

---data : <class 'dict'>


In [165]:
model.predict_proba(indiv_1.iloc[lambda x: x.index == 100003][col])

array([[0.19710162, 0.80289838]])

In [84]:
probability.to_list()

AttributeError: 'str' object has no attribute 'to_list'

In [96]:
probability

'[[0.09307385 0.90692615]]'

In [85]:
probability.replace(' ',',')

'['

In [190]:
list(eval(probability.strip('[[]]').replace(' ', ',')))

[0.09307385, 0.90692615]

In [100]:
pd.DataFrame(list(eval(probability.strip('[[]]').replace(' ', ',')))).T


,0,1
0,0.093074,0.906926


In [191]:
data = 	{
  "prediction": "1",
  "probability": "[[0.09307385 0.90692615]]"
}

In [192]:
probability = list(
    eval(data['probability'].strip('[[]]').replace(' ', ',')))
probability = pd.DataFrame(probability).T

In [222]:
probability.iloc[0][1]

0.90692615

In [136]:
eval(data['probability'].split(' '))  #.strip('\'')


TypeError: eval() arg 1 must be a string, bytes or code object

In [111]:
data = [[0.09307385, 0.90692615]]

In [109]:
data[0][0]

0.09307385

In [16]:
X = ((indiv_1.iloc[lambda x: x.index == 100004][col]).to_dict())

result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [17]:
X = (indiv_1.iloc[lambda x: x.index == 100002][col].to_dict())

result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [18]:
X = (indiv_1.iloc[lambda x: x.index == 100003][col].to_dict())
result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [22]:
X = (indiv_1.iloc[lambda x: x.index == 100031][col].to_dict())
result = request_prediction(LRSMOTE_URI, X)
result.content

---data : <class 'dict'>


b'{"prediction":"1","probability":"[[0.09307385 0.90692615]]"}'

In [11]:
df = pd.read_csv('../API/ocp7smlt/new_train_application.csv')
df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
from statistics import median

sp = SimpleImputer(missing_values=np.nan, strategy='median', verbose=1)

result_imputed = sp.fit_transform(df)
feature_keeped = list(sp.get_feature_names_out())

df = pd.DataFrame(result_imputed, columns=feature_keeped)


/home/moun_j/.pyenv/versions/3.8.12/lib/python3.8/site-packages/sklearn/impute/_base.py:525: UserWarning: Skipping features without any observed values: ['BURO_MONTHS_BALANCE_MIN_MIN' 'BURO_MONTHS_BALANCE_MAX_MAX'
 'BURO_MONTHS_BALANCE_SIZE_MEAN' 'BURO_STATUS_0_MEAN_MEAN'
 'BURO_STATUS_1_MEAN_MEAN' 'BURO_STATUS_2_MEAN_MEAN'
 'BURO_STATUS_3_MEAN_MEAN' 'BURO_STATUS_C_MEAN_MEAN'
 'BURO_STATUS_X_MEAN_MEAN' 'BURO_STATUS_nan_MEAN_MEAN'
 'ACTIVE_MONTHS_BALANCE_MIN_MIN' 'ACTIVE_MONTHS_BALANCE_MAX_MAX'
 'ACTIVE_MONTHS_BALANCE_SIZE_MEAN' 'CLOSED_MONTHS_BALANCE_MIN_MIN'
 'CLOSED_MONTHS_BALANCE_MAX_MAX' 'CLOSED_MONTHS_BALANCE_SIZE_MEAN']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [ ]:
# list(sp.get_feature_names_out())

In [17]:
df.isna().sum()

SK_ID_CURR                          0
TARGET                              0
CODE_GENDER                         0
FLAG_OWN_CAR                        0
FLAG_OWN_REALTY                     0
                                   ..
CC_NAME_CONTRACT_STATUS_nan_MAX     0
CC_NAME_CONTRACT_STATUS_nan_MEAN    0
CC_NAME_CONTRACT_STATUS_nan_SUM     0
CC_NAME_CONTRACT_STATUS_nan_VAR     0
CC_COUNT                            0
Length: 755, dtype: int64

In [34]:
df.to_csv('../API/ocp7smlt/new_train_cleaned_application.csv', index=False)

In [19]:
df.dtypes

SK_ID_CURR                          float64
TARGET                              float64
CODE_GENDER                         float64
FLAG_OWN_CAR                        float64
FLAG_OWN_REALTY                     float64
                                     ...   
CC_NAME_CONTRACT_STATUS_nan_MAX     float64
CC_NAME_CONTRACT_STATUS_nan_MEAN    float64
CC_NAME_CONTRACT_STATUS_nan_SUM     float64
CC_NAME_CONTRACT_STATUS_nan_VAR     float64
CC_COUNT                            float64
Length: 755, dtype: object

In [56]:
(abs(df.DAYS_BIRTH/360))

0        26.280556
1        46.569444
2        52.905556
3        52.791667
4        55.366667
           ...    
19995    30.041667
19996    39.902778
19997    39.611111
19998    42.958333
19999    40.008333
Name: DAYS_BIRTH, Length: 20000, dtype: float64

In [46]:
def feature_importance(df):
    feature_names = df.drop(columns=['TARGET', 'SK_ID_CURR']).columns
    forest = RandomForestClassifier(random_state=0)
    forest.fit(df.drop(columns=['TARGET', 'SK_ID_CURR']), df['TARGET'])

    forest_importances = pd.Series(
        forest.feature_importances_,
        index=feature_names).sort_values(ascending=False)[:20]

    # pd.DataFrame(
    #     forest_importances,
    #     columns=['importance']).reset_index().rename(columns={
    #         'index': 'feature'
    #     }).to_csv(params.PATH_OUTPUT + 'feature_selection_application.csv',
    #               index=False)
    return pd.DataFrame(
        forest_importances,
        columns=['importance']).reset_index().rename(columns={
            'index': 'feature'
        }) #list(forest_importances[:10].index)

In [47]:
feat_imp = feature_importance(df)

In [50]:
list(feat_imp['feature'][:20])

['EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'DAYS_BIRTH',
 'DAYS_ID_PUBLISH',
 'PAYMENT_RATE',
 'DAYS_REGISTRATION',
 'INCOME_CREDIT_PERC',
 'ANNUITY_INCOME_PERC',
 'AMT_ANNUITY',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_EMPLOYED',
 'EXT_SOURCE_1',
 'INCOME_PER_PERSON',
 'DAYS_EMPLOYED_PERC',
 'AMT_CREDIT',
 'REGION_POPULATION_RELATIVE',
 'AMT_INCOME_TOTAL',
 'AMT_GOODS_PRICE',
 'HOUR_APPR_PROCESS_START',
 'AMT_REQ_CREDIT_BUREAU_YEAR']

In [ ]:
# Affichage des features importantes
def impPlot(imp, name):
    figure = px.bar(imp,
                    x=imp.values,
                    y=imp.keys(),
                    labels={
                        'x': 'Importance Value',
                        'index': 'Columns'
                    },
                    text=np.round(imp.values, 2),
                    title=name + ' Feature Selection Plot',
                    width=1000,
                    height=600)
    figure.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    st.plotly_chart(figure)

In [12]:
df_app = pd.read_csv('../API/ocp7smlt/new_train_cleaned_application.csv')
df_app.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002.0,1.0,0.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003.0,0.0,1.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100004.0,0.0,0.0,1.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,100006.0,0.0,1.0,0.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007.0,0.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
df_app.columns

Index(['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'CC_NAME_CONTRACT_STATUS_Signed_MAX',
       'CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'CC_NAME_CONTRACT_STATUS_Signed_VAR', 'CC_NAME_CONTRACT_STATUS_nan_MIN',
       'CC_NAME_CONTRACT_STATUS_nan_MAX', 'CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'CC_NAME_CONTRACT_STATUS_nan_SUM', 'CC_NAME_CONTRACT_STATUS_nan_VAR',
       'CC_COUNT'],
      dtype='object', length=755)

In [14]:
df_app.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002.0,1.0,0.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003.0,0.0,1.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,100004.0,0.0,0.0,1.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,100006.0,0.0,1.0,0.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007.0,0.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
LRSMOTE_URI = 'https://ocp7gitapi.herokuapp.com/predict'

In [16]:
def request_prediction(model_uri, data):
    '''Réalise une prédiction à partur de l'API hébergée sur Heroku'''
    response = {}
    headers = {"Content-Type": "application/json"}

    data_json = {'data': data.to_json()}

    response = requests.request(method='POST',
                                headers=headers,
                                url=model_uri,
                                json=data_json)

    if response.status_code != 200:
        raise Exception("Request failed with status {}, {}".format(
            response.status_code, response.text))

    return response

In [17]:
result = json.loads(request_prediction(LRSMOTE_URI, X3).content)


NameError: name 'X3' is not defined

# Représentation du client dans son ensemble

In [22]:
result = model.predict(df_app[col])

In [23]:
result

array([1, 0, 0, ..., 0, 0, 0])

In [24]:
df_app['predicted'] = result

In [25]:
df_app[df_app['predicted'] == 1][['SK_ID_CURR','TARGET','predicted']]


,SK_ID_CURR,TARGET,predicted
0,100002.0,1.0,1
101,100120.0,0.0,1
110,100130.0,1.0,1
128,100149.0,0.0,1
163,100192.0,1.0,1
...,...,...,...
19745,123022.0,0.0,1
19841,123141.0,1.0,1
19879,123184.0,0.0,1
19912,123223.0,0.0,1


In [26]:
def result_prediction(target, predicted):
    if predicted == 1:
        if int(target) == 1:
            pred = 'True Positive'
        else:
            pred = 'False Positive'
    else:
        if int(target) == 1:
            pred = 'False Negative)'
        else:
            pred = 'True Negative)'
    return pred

In [27]:
df_app['result'] = (df_app['TARGET'] * 2 - df_app['predicted'])


In [34]:
df_app['result'].value_counts()

TN    18228
FN     1511
FP      194
TP       67
Name: result, dtype: int64

In [29]:
cat_map = {-1 : "FP", 0 : "TN", 1 : "TP", 2:"FN"}
# s = pd.Series(cat_map)
# df_app = df_app['result'].map(s)
df_app['result'] = df_app['result'].map(cat_map)

In [30]:
type(df_app)

pandas.core.frame.DataFrame

In [35]:
pie_data = df_app.groupby(result)

In [31]:
import plotly.express as px

In [37]:
pie_data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT,predicted,result
0,100002.0,1.0,0.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,TP
1,100003.0,0.0,1.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,TN
2,100004.0,0.0,0.0,1.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,TN
3,100006.0,0.0,1.0,0.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,TN
4,100007.0,0.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,TN
5,100008.0,0.0,0.0,0.0,0.0,0.0,99000.0,490495.5,27517.5,454500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,TN
101,100120.0,0.0,1.0,0.0,1.0,0.0,135000.0,781920.0,28215.0,675000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,FP
110,100130.0,1.0,1.0,0.0,0.0,1.0,157500.0,723996.0,30802.5,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,TP
128,100149.0,0.0,0.0,1.0,1.0,0.0,171000.0,539590.5,26374.5,445500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,FP
163,100192.0,1.0,1.0,0.0,1.0,0.0,111915.0,225000.0,21037.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,TP


In [33]:
px.pie(pie_data, names='result', values='TARGET', title='repartition')